In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import glob
import json
from pathlib import Path

In [2]:
def load_json(dir):
    try:
        with open(dir) as f:
            data = json.load(f)
            
        return data
    
    except:
        return None

In [3]:
RESULT_DIR = "..\\result_summary"

In [4]:
def parse_file_nm(file_nm):
    mcts_info = None
    
    components = file_nm.split("\\")
    
    if "debug" in components:
        components.remove("debug")
    
    method = components[2]
    
    diff_cut = 1
    
    if 'am' in method:
        _, _, method, _, prob_type, num_probs, model_info, baseline_info, _ = components
        
    elif 'mcts' in method:
        # print(components)
        _, _, method, diff_cut, _, prob_type, num_probs, model_info, baseline_info, mcts_info, _ = components
        diff_cut = float(diff_cut.split("-")[-1])

    num_probs = num_probs.split("-")[0]
    num_probs = int(num_probs.split("_")[1])
    baseline_info = baseline_info.split("-")[-1]
    
    return method, prob_type, num_probs, model_info, baseline_info, mcts_info, diff_cut

In [5]:
for file in glob.glob(f"{RESULT_DIR}/**/*.json", recursive=True):
    result = load_json(file)

    avg_result = False
    for k, v in result.items():
        for k2, v2 in v.items():
            if 'result' in k2:
                avg_result = True
                break
    
    if not avg_result and len(result) != 102:
        print(len(result))
        print(file)
        
# need to do diff-0.75 and CVRP with N=50 for 4 and 6

66
..\result_summary\mcts\diff-1\pretrained_result\cvrp\N_100-B_64\shared_mha-128-4-32-4-relu-10-0.0001\1562-1-mean\ns_100-temp_5-cpuct_1.1-norm_True-rollout_False-ec_0.0100\epoch=149-train_score=16.01232.json
66
..\result_summary\mcts\diff-1\pretrained_result\cvrp\N_100-B_64\shared_mha-128-4-32-4-relu-10-0.0001\1562-1-mean\ns_100-temp_5-cpuct_1.1-norm_True-rollout_False-ec_0.0100\epoch=199-train_score=16.35231.json
66
..\result_summary\mcts\diff-1\pretrained_result\cvrp\N_100-B_64\shared_mha-128-4-32-4-relu-10-0.0001\1562-1-mean\ns_100-temp_5-cpuct_1.1-norm_True-rollout_False-ec_0.0100\epoch=249-train_score=15.99893.json
66
..\result_summary\mcts\diff-1\pretrained_result\cvrp\N_100-B_64\shared_mha-128-4-32-4-relu-10-0.0001\1562-1-mean\ns_100-temp_5-cpuct_1.1-norm_True-rollout_False-ec_0.0100\epoch=299-train_score=15.82023.json
66
..\result_summary\mcts\diff-1\pretrained_result\cvrp\N_100-B_64\shared_mha-128-4-32-4-relu-10-0.0001\1562-1-mean\ns_100-temp_5-cpuct_1.1-norm_True-rollout_Fa

In [53]:
# parse all json file named with "all_result_avg.json" from the RESULT_DIR

all_result = {}

for file in glob.glob(f"{RESULT_DIR}/**/all_result_avg.json", recursive=True):
    if 'cross_test_result' in file:
        continue
    
    if 'first_second_diff' in file:
        continue
    
    data = load_json(file)
    # print(file)
    method, prob_type, num_probs, model_info, baseline_info, mcts_info, diff_cut = parse_file_nm(file)
    
    if data is not None:
        
        if method not in all_result:
            all_result[method] = {}
        
        if prob_type not in all_result[method]:
            all_result[method][prob_type] = {}
            
        if num_probs not in all_result[method][prob_type]:
            all_result[method][prob_type][num_probs] = {}
        
        if mcts_info is None:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
            
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = data
                
        else:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
                
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = {}
                
            if mcts_info not in all_result[method][prob_type][num_probs][model_info][baseline_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info] = {}
                
            if diff_cut not in all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info][diff_cut] = data
                
    # break

In [54]:
def get_parital_result(method, prob_type, num_prob):       
    partial_result = {'model_info': [], 'baseline': [], 'mcts_info': [], 'epoch': [], 'score': [], 'score_std': [], 'runtime': [],  'runtime_std': [], 'diff_cut': []}
    
    def _append_epoch_result(model_info, baseline_info, epoch, mcts, epoch_score, epoch_score_std, epoch_runtime, epoch_runtime_std, diff_cut):
        partial_result['model_info'].append(model_info)
        partial_result['baseline'].append(baseline_info)
        partial_result['mcts_info'].append(mcts)
        partial_result['epoch'].append(epoch)                
        partial_result['score'].append(epoch_score)
        partial_result['score_std'].append(epoch_score_std)
        partial_result['runtime'].append(epoch_runtime)
        partial_result['runtime_std'].append(epoch_runtime_std)
        partial_result['diff_cut'].append(diff_cut)

    target_result = all_result[method][prob_type][num_prob]
    
    if "am" in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_epoch_results = target_result[model_info][baseline_info]
                
                for epoch in all_epoch_results:
                    current_epoch_result = all_epoch_results[epoch]['result_avg']
                    epoch_score = current_epoch_result['score']
                    epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                    epoch_runtime = current_epoch_result['runtime']
                    epoch_runtime_std = all_epoch_results[epoch]['result_std']['runtime']
                    
                    _append_epoch_result(model_info, baseline_info, epoch, "am", epoch_score,epoch_score_std, epoch_runtime, epoch_runtime_std, 1)
                    
    elif 'mcts' in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_mcts_results = target_result[model_info][baseline_info]
                
                for mcts_info in all_mcts_results:
                    all_diff_results = all_mcts_results[mcts_info]
                    
                    for diff_cut in all_diff_results:
                        all_epoch_results = all_diff_results[diff_cut]
                    
                        for epoch in all_epoch_results:
                            current_epoch_result = all_epoch_results[epoch]['result_avg']
                            epoch_score = current_epoch_result['score']
                            epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                            epoch_runtime = current_epoch_result['runtime']
                            epoch_runtime_std = all_epoch_results[epoch]['result_std']['runtime']
                            
                            _append_epoch_result(model_info, baseline_info, epoch, mcts_info, epoch_score,epoch_score_std, epoch_runtime, epoch_runtime_std, diff_cut)                                         
            
    df = pd.DataFrame.from_dict(partial_result)
    df['activation'] = df['model_info'].apply(lambda x: x.split("-")[5])
    df['enc_layers'] = df['model_info'].apply(lambda x: x.split('-')[2])
    df = df.drop(columns=['model_info'])
    
    df['train_score'] = df.apply(lambda x: x['epoch'].split("-")[1].split("=")[1], axis=1).astype(float)
    df['epoch'] =  df.apply(lambda x: x['epoch'].split("-")[0].split("=")[1], axis=1).astype(int)
    
    if 'mcts' in method:
        df['cpuct'] = df['mcts_info'].apply(lambda x: x.split("-")[2].split('_')[1])
        df['ns'] = df['mcts_info'].apply(lambda x: x.split("-")[0].split('_')[1])
        df['cpuct'] = df['cpuct'].astype(float)
        df['ns'] = df['ns'].astype(int)
    
    elif 'am' in method:
        df['cpuct'] = 0
        df['ns'] = 0
        
    df = df.drop(columns=['mcts_info'])
    

        
    return df

In [55]:
def plot_bar_result(base_df, baseline, activation, prob_type, num_prob, plot_dev=False, hue='cpuct'):
    if prob_type == 'tsp':
        y_ranges = {20: (3.75, 3.95), 50: (5.7, 5.875), 100: (7.95, 8.25)}
        
    elif prob_type == 'cvrp':
        y_ranges = {20: (6.1, 7.5), 50: (9, 12), 100: (7.95, 8.25)}
        
    _df = base_df[(base_df['baseline'] == baseline) & (base_df['activation'] == activation)]
    
    # drop rows where the mcts_info is 0.8
    _df = _df[_df['cpuct'] != '0.8']
    
    # sort by the train score
    _df = _df.sort_values(by=['epoch', 'cpuct', 'ns']).reset_index(drop=True)
    _df['score_std'] = _df['score_std'].astype(float)
    
    _df['cpuct'] = _df['cpuct'].astype(object)
    _df['cpuct'] = _df['cpuct'].fillna('am')
    
    _df['ns'] = _df['ns'].astype(object)
    _df['ns'] = _df['ns'].fillna('am')
    
    plt.figure(figsize=(12, 7))
    
    ax = sns.barplot(data=_df, x=_df['epoch'], y='score', hue=hue)
    
    if plot_dev:
        bar_coords = [(rect.get_x(), rect.get_y(), rect.get_width(), rect.get_height()) for rect in ax.patches]
                
        for i, (x, y, w, h) in enumerate(bar_coords):
            x_pos = x + w / 2
            y_top = y + h + 0.01
            
            plt.errorbar(x=x_pos, y=y_top, yerr=_df['score_std'][i], fmt='none', color='black', capsize=4)
    
    plt.title(f"{prob_type}_{num_prob}_{baseline}_{activation}")
    plt.ylim(*y_ranges[num_prob])
    plt.legend()
    
    path = Path(f"../result_image/bars")
    
    if not path.exists():
        path.mkdir(parents=True)
        
    
    plt.savefig(f"{path}/{prob_type}_{num_prob}_{baseline}_{activation}.png")
    plt.show()

In [56]:
# tsp_20 = pd.concat([get_parital_result('am', 'tsp', 20), get_parital_result('mcts_v2', 'tsp', 20)])

# for _baseline in ['mean', 'val']:
#     for _activation in ['relu', 'swiglu']:
#         plot_bar_result(tsp_20, _baseline, _activation, 'tsp', 20, hue='cpuct')

In [57]:
tsp_100 = pd.concat([get_parital_result('am', 'tsp', 100), get_parital_result('mcts', 'tsp', 100)])
tsp_100 = tsp_100[(tsp_100['epoch']==299) & (tsp_100['ns']!=250) & (tsp_100['activation'] == 'swiglu') & (tsp_100['ns']>0)]
tsp_100.groupby(['diff_cut', 'ns', 'baseline', 'epoch']).mean()
tsp_100.groupby(['diff_cut']).mean()

C:\Users\admin\AppData\Local\Temp\ipykernel_18312\4166975296.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tsp_100.groupby(['diff_cut', 'ns', 'baseline', 'epoch']).mean()
C:\Users\admin\AppData\Local\Temp\ipykernel_18312\4166975296.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tsp_100.groupby(['diff_cut']).mean()


,epoch,score,score_std,runtime,runtime_std,train_score,cpuct,ns
diff_cut,,,,,,,,
0.25,299.0,7.997735,0.082563,0.456120,0.043028,7.861817,1.1,533.333333
0.50,299.0,7.996345,0.082692,0.647252,0.883964,7.861817,1.1,533.333333
0.75,299.0,7.999055,0.083382,6.447182,13.155217,7.861817,1.1,533.333333
1.00,299.0,8.153958,0.103508,215.137975,1474.669240,7.861817,1.1,533.333333


In [58]:
tsp_100 = pd.concat([get_parital_result('am', 'tsp', 100), get_parital_result('mcts', 'tsp', 100)])
tsp_100.to_csv("../result_csv/tsp_100.csv")
tsp_100.sort_values(by=['score']).head(15)

,baseline,epoch,score,score_std,runtime,runtime_std,diff_cut,activation,enc_layers,train_score,cpuct,ns
127,val,99,7.829545,0.000000,287.198189,0.000000,1.00,relu,4,7.96576,1.1,1000
126,val,149,7.858331,0.000000,316.855922,0.000000,1.00,relu,4,7.95501,1.1,1000
130,val,49,7.909483,0.000000,327.850042,0.000000,1.00,relu,4,8.01927,1.1,1000
128,val,299,7.948353,0.000000,396.697101,0.000000,1.00,relu,4,7.85948,1.1,1000
624,val,299,7.952167,0.072568,10.248525,25.766765,0.75,swiglu,6,7.86777,1.1,1000
516,mean,299,7.952278,0.073115,1.087797,0.199681,0.75,swiglu,6,7.90623,1.1,100
657,val,299,7.952654,0.072702,0.581626,0.353638,0.50,swiglu,6,7.86777,1.1,500
622,val,299,7.952794,0.072653,0.929683,1.953379,0.50,swiglu,6,7.86777,1.1,1000
577,mean,299,7.952995,0.073776,4.318947,5.818327,0.75,swiglu,6,7.90623,1.1,500
645,val,299,7.953113,0.072455,0.438101,0.087440,0.50,swiglu,6,7.86777,1.1,250


In [59]:
for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        df = pd.concat([get_parital_result('am', env_type, N), get_parital_result('mcts', env_type, N)])
        df.to_csv(f"../result_csv/{env_type}_{N}.csv", index=False)

In [60]:
df

,baseline,epoch,score,score_std,runtime,runtime_std,diff_cut,activation,enc_layers,train_score,cpuct,ns
0,mean,49,16.717783,3.231875,0.471543,0.000779,1.0,relu,4,16.39185,0.0,0
1,mean,99,16.606976,3.071906,0.473988,0.001013,1.0,relu,4,16.41725,0.0,0
2,mean,149,16.574835,2.968623,0.479116,0.001992,1.0,relu,4,16.01232,0.0,0
3,mean,199,16.535594,3.113413,0.480392,0.001704,1.0,relu,4,16.35231,0.0,0
4,mean,249,16.490532,3.008314,0.476007,0.000952,1.0,relu,4,15.99893,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
485,val,199,16.050198,3.403988,157.990522,67.594072,1.0,swiglu,6,16.17082,1.1,500
486,val,299,16.005585,3.401209,163.949837,150.163978,1.0,swiglu,6,16.26320,1.1,500
487,val,149,16.177041,2.983011,164.035912,74.989390,1.0,swiglu,6,16.41855,1.1,500
488,val,249,16.069751,2.970252,162.490093,136.549853,1.0,swiglu,6,16.53491,1.1,500


In [61]:
# load tsp and N=20

def get_pivot_table(env_type, N, epoch=299, enc_layers=6, diff_cut=0.75, index=['epoch', 'ns', 'activation', 'baseline'], refine_to_str=True):
    df = pd.read_csv(f"../result_csv/{env_type}_{N}.csv")
    cond1 = df['epoch'] == epoch
    cond2 = df['enc_layers'] == enc_layers
    cond3 = df['diff_cut'] == diff_cut 
    cond4 = df['diff_cut'] == 1
    df = df[cond1 & cond2 & (cond3 | cond4)]
    pivot_table = df.pivot_table(index=index, values=['score', 'score_std', 'runtime', 'runtime_std'])
    # # reindex the column order as to score mean, score std, runtime mean, runtime std
    pivot_table = pivot_table.reindex(columns=['score', 'score_std', 'runtime', 'runtime_std'])
    if refine_to_str:
        pivot_table['score'] = pivot_table.apply(lambda x: f"{round(x['score'], 5):.4f} ± {round(x['score_std'], 2):.2f}", axis=1)
        pivot_table = pivot_table.drop(['score_std'], axis=1)
    
        pivot_table['runtime'] = pivot_table.apply(lambda x: f"{round(x['runtime'], 4):.3f} ± {round(x['runtime_std'], 2):.2f}", axis=1)
        pivot_table = pivot_table.drop(['runtime_std'], axis=1)
        
    return pivot_table

In [62]:
tsp_results = [get_pivot_table('tsp', N, refine_to_str=False) for N in [20, 50, 100]]
all_results_df = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                                   \
                                   score score_std   runtime runtime_std   
epoch ns   activation baseline                                             
299   0    relu       mean      3.849166  0.087340  0.106683    0.000164   
                      val       3.848943  0.084740  0.111188    0.000175   
           swiglu     mean      3.846409  0.084434  0.109879    0.000128   
                      val       3.848185  0.085138  0.112462    0.000178   
      100  relu       mean      3.813529  0.070737  1.227124    0.060662   
                      val       3.817411  0.070527  1.214962    0.039518   
           swiglu     mean      3.813522  0.069293  0.972194    0.498330   
                      val       3.813971  0.069458  1.120984    0.258571   
      500  relu       mean      3.813752  0.070686  2.210185    0.094628   
                      val       3.807701  0.073040  2.201901    0.090871   
           swiglu     mean      3.813456  0.069258  1.963953    0.112107   
                      val       3.814215  0.069477  2.284253    0.327571   
      1000 relu       mean      3.813752  0.070686  3.738607    0.227895   
                      val       3.807701  0.073040  3.724565    0.251329   
           swiglu     mean      3.813495  0.069279  3.491623    0.175182   
                      val       3.814215  0.069477  3.808314    0.342911   

                                     50                                    \
                                   score score_std    runtime runtime_std   
epoch ns   activation baseline                                              
299   0    relu       mean      5.736106  0.059724   0.231187    0.000463   
                      val       5.738576  0.063584   0.229614    0.000554   
           swiglu     mean      5.726738  0.069453   0.237449    0.000701   
                      val       5.740503  0.066726   0.233933    0.000357   
      100  relu       mean      5.743749  0.061082   8.448053    3.821418   
                      val       5.743931  0.065241   8.492847    3.454599   
           swiglu     mean      5.729036  0.065645   8.753800    2.880461   
                      val       5.752099  0.065210   8.142497    2.833176   
      500  relu       mean      5.740502  0.061935  22.108644   49.918029   
                      val       5.739597  0.064672  19.934267   33.121937   
           swiglu     mean      5.730324  0.066231  19.393686   43.397223   
                      val       5.750293  0.063566  21.155173   41.275611   
      1000 relu       mean      5.739415  0.061390  33.966379  125.769903   
                      val       5.740247  0.064062  30.358330   71.954300   
           swiglu     mean      5.730197  0.066255  30.292925  126.293590   
                      val       5.750130  0.064603  32.241153  109.827326   

                                     100                                     
                                   score score_std     runtime  runtime_std  
epoch ns   activation baseline                                               
299   0    relu       mean      7.985253  0.078364    0.449912     0.000449  
                      val       8.048133  0.079650    0.450653     0.000271  
           swiglu     mean      7.956173  0.073098    0.454370     0.000451  
                      val       7.955108  0.072696    0.456002     0.000474  
      100  relu       mean      8.009914  0.096513   25.587843     9.278344  
                      val       8.062968  0.096528   27.543703     2.027910  
           swiglu     mean      7.972971  0.082268   26.325069     6.588629  
                      val       8.015496  0.098413   27.053944     3.627509  
      500  relu       mean      8.011700  0.099405  103.089253   314.895483  
                      val       8.058749  0.090377  106.338934   317.862601  
           swiglu     mean      7.974341  0.081985  101.822795   354.816920  
                      val       8.00

In [63]:
tsp_results = [get_pivot_table('tsp', N) for N in [20, 50, 100]]
all_results_df = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                           50   \
                                        score       runtime          score   
epoch ns   activation baseline                                               
299   0    relu       mean      3.8492 ± 0.09  0.107 ± 0.00  5.7361 ± 0.06   
                      val       3.8489 ± 0.08  0.111 ± 0.00  5.7386 ± 0.06   
           swiglu     mean      3.8464 ± 0.08  0.110 ± 0.00  5.7267 ± 0.07   
                      val       3.8482 ± 0.09  0.113 ± 0.00  5.7405 ± 0.07   
      100  relu       mean      3.8135 ± 0.07  1.227 ± 0.06  5.7438 ± 0.06   
                      val       3.8174 ± 0.07  1.215 ± 0.04  5.7439 ± 0.07   
           swiglu     mean      3.8135 ± 0.07  0.972 ± 0.50  5.7290 ± 0.07   
                      val       3.8140 ± 0.07  1.121 ± 0.26  5.7521 ± 0.07   
      500  relu       mean      3.8138 ± 0.07  2.210 ± 0.09  5.7405 ± 0.06   
                      val       3.8077 ± 0.07  2.202 ± 0.09  5.7396 ± 0.06   
           swiglu     mean      3.8135 ± 0.07  1.964 ± 0.11  5.7303 ± 0.07   
                      val       3.8142 ± 0.07  2.284 ± 0.33  5.7503 ± 0.06   
      1000 relu       mean      3.8138 ± 0.07  3.739 ± 0.23  5.7394 ± 0.06   
                      val       3.8077 ± 0.07  3.725 ± 0.25  5.7402 ± 0.06   
           swiglu     mean      3.8135 ± 0.07  3.492 ± 0.18  5.7302 ± 0.07   
                      val       3.8142 ± 0.07  3.808 ± 0.34  5.7501 ± 0.06   

                                                           100  \
                                        runtime          score   
epoch ns   activation baseline                                   
299   0    relu       mean         0.231 ± 0.00  7.9852 ± 0.08   
                      val          0.230 ± 0.00  8.0481 ± 0.08   
           swiglu     mean         0.237 ± 0.00  7.9562 ± 0.07   
                      val          0.234 ± 0.00  7.9551 ± 0.07   
      100  relu       mean         8.448 ± 3.82  8.0099 ± 0.10   
                      val          8.493 ± 3.45  8.0630 ± 0.10   
           swiglu     mean         8.754 ± 2.88  7.9730 ± 0.08   
                      val          8.143 ± 2.83  8.0155 ± 0.10   
      500  relu       mean       22.109 ± 49.92  8.0117 ± 0.10   
                      val        19.934 ± 33.12  8.0587 ± 0.09   
           swiglu     mean       19.394 ± 43.40  7.9743 ± 0.08   
                      val        21.155 ± 41.28  8.0079 ± 0.07   
      1000 relu       mean      33.966 ± 125.77  8.0077 ± 0.10   
                      val        30.358 ± 71.95  8.0510 ± 0.09   
           swiglu     mean      30.293 ± 126.29  7.9779 ± 0.08   
                      val       32.241 ± 109.83  7.9900 ± 0.08   

                                                   
                                          runtime  
epoch ns   activation baseline                     
299   0    relu       mean           0.450 ± 0.00  
                      val            0.451 ± 0.00  
           swiglu     mean           0.454 ± 0.00  
                      val            0.456 ± 0.00  
      100  relu       mean          25.588 ± 9.28  
                      val           27.544 ± 2.03  
           swiglu     mean          26.325 ± 6.59  
                      val           27.054 ± 3.63  
      500  relu       mean       103.089 ± 314.90  
                      val        106.339 ± 317.86  
           swiglu     mean       101.823 ± 354.82  
                      val        105.429 ± 403.83  
      1000 relu       mean      181.956 ± 1789.51  
                      val       183.703 ± 1847.25  
           swiglu     mean      178.833 ± 2319.60  
                      val       186.502 ± 2382.77

In [64]:
cvrp_results = [get_pivot_table('cvrp', N) for N in [20, 50, 100]]
all_results_df = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                            50   \
                                        score       runtime           score   
epoch ns   activation baseline                                                
299   0    relu       mean      6.4097 ± 0.75  0.131 ± 0.00  10.8050 ± 1.65   
                      val       6.4553 ± 0.80  0.132 ± 0.00  10.8634 ± 1.65   
           swiglu     mean      6.4231 ± 0.73  0.133 ± 0.00  10.7940 ± 1.70   
                      val       6.4201 ± 0.72  0.132 ± 0.00  10.8402 ± 1.48   
      100  relu       mean      6.3968 ± 0.74  1.363 ± 0.37  10.8999 ± 1.68   
                      val       6.4738 ± 0.77  1.438 ± 0.30  10.9204 ± 1.70   
           swiglu     mean      6.4221 ± 0.69  1.505 ± 0.45  10.8693 ± 1.76   
                      val       6.4151 ± 0.67  1.442 ± 0.51  10.9194 ± 1.59   
      500  relu       mean      6.4078 ± 0.76  3.873 ± 1.73  10.8981 ± 1.69   
                      val       6.5081 ± 0.74  4.312 ± 2.12  10.9336 ± 1.76   
           swiglu     mean      6.4342 ± 0.69  3.993 ± 2.22  10.8806 ± 1.78   
                      val       6.4269 ± 0.66  4.026 ± 1.79  10.9101 ± 1.54   
      1000 relu       mean      6.4384 ± 0.75  6.616 ± 4.23  10.8814 ± 1.72   
                      val       6.5068 ± 0.73  6.780 ± 3.74  10.9336 ± 1.74   
           swiglu     mean      6.4381 ± 0.70  6.550 ± 3.44  10.8796 ± 1.75   
                      val       6.4341 ± 0.67  6.774 ± 3.58  10.9181 ± 1.58   
      250  relu       mean                NaN           NaN  10.8228 ± 1.61   
                      val                 NaN           NaN  10.8858 ± 1.62   
           swiglu     mean                NaN           NaN  10.7991 ± 1.75   
                      val                 NaN           NaN  10.8455 ± 1.49   

                                                            100  \
                                        runtime           score   
epoch ns   activation baseline                                    
299   0    relu       mean         0.279 ± 0.00  16.4418 ± 2.99   
                      val          0.276 ± 0.00  16.4463 ± 3.00   
           swiglu     mean         0.278 ± 0.00  16.4043 ± 3.09   
                      val          0.277 ± 0.00  16.3575 ± 3.04   
      100  relu       mean         7.491 ± 1.31  16.3123 ± 3.32   
                      val          6.920 ± 1.68  16.2629 ± 3.09   
           swiglu     mean         6.932 ± 1.27  16.2225 ± 3.18   
                      val          7.078 ± 2.04  16.2489 ± 3.26   
      500  relu       mean       25.497 ± 36.98  16.3137 ± 3.14   
                      val        25.497 ± 38.86  16.2650 ± 3.14   
           swiglu     mean       24.775 ± 41.08  16.2238 ± 3.16   
                      val        27.481 ± 25.40  16.1766 ± 3.23   
      1000 relu       mean      45.574 ± 136.87  16.2834 ± 3.19   
                      val       46.212 ± 149.18  16.2638 ± 3.12   
           swiglu     mean      45.049 ± 160.53  16.2088 ± 3.24   
                      val       49.487 ± 113.46  16.1746 ± 3.19   
      250  relu       mean         2.046 ± 0.96             NaN   
                      val          2.090 ± 1.02             NaN   
           swiglu     mean         2.008 ± 1.14             NaN   
                      val          2.271 ± 1.31             NaN   

                                                  
                                         runtime  
epoch ns   activation baseline                    
299   0    relu       mean          0.523 ± 0.00  
                      val           0.525 ± 0.00  
           swiglu     mean          0.527 ± 0.00  
                      val           0.525 ± 0.00  
      100  relu       mean         18.919 ± 3.11  
                      val          19.441 ± 1.57  
           swiglu     mean        19.299 ± 15.57  
                      val          19.363 ± 2.14  
      500  relu       mean        85.155 ± 93.30  
                      val         86.026 ± 69.93  
           swiglu  

In [65]:
tsp_results = [get_pivot_table('tsp', N, index=['activation']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50                   \
                         score       runtime           score         runtime   
     activation                                                                
tsp  relu        3.8176 ± 0.07  2.061 ± 0.11   5.7407 ± 0.06  17.648 ± 41.15   
     swiglu      3.8186 ± 0.07  1.965 ± 0.24   5.7394 ± 0.07  17.174 ± 46.64   
cvrp relu        6.4520 ± 0.75  3.502 ± 1.78  10.8944 ± 1.69  19.942 ± 45.73   
     swiglu      6.4274 ± 0.69  3.489 ± 1.71  10.8771 ± 1.65  20.402 ± 43.12   

                            100                   
                          score          runtime  
     activation                                   
tsp  relu         8.0312 ± 0.09  89.810 ± 611.55  
     swiglu       7.9849 ± 0.08  89.489 ± 781.60  
cvrp relu        16.3065 ± 3.14  80.219 ± 118.95  
     swiglu      16.2337 ± 3.19  79.992 ± 121.70

In [66]:
tsp_results = [get_pivot_table('tsp', N, index=['activation'], refine_to_str=False) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation'], refine_to_str=False) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                                         50   \
                    score score_std   runtime runtime_std      score   
     activation                                                        
tsp  relu        3.817557  0.073537  2.060897    0.109296   5.740683   
     swiglu      3.818596  0.071575  1.964642    0.244975   5.739386   
cvrp relu        6.452016  0.751824  3.501845    1.783401  10.894437   
     swiglu      6.427443  0.688956  3.489025    1.712907  10.877050   

                                                        100            \
                score_std    runtime runtime_std      score score_std   
     activation                                                         
tsp  relu        0.062862  17.648417   41.148671   8.031242  0.092655   
     swiglu      0.065657  17.173561   46.643988   7.984901  0.080620   
cvrp relu        1.694117  19.942269   45.733298  16.306458  3.140024   
     swiglu      1.652815  20.402460   43.124919  16.233701  3.188385   

                                        
                   runtime runtime_std  
     activation                         
tsp  relu        89.809793  611.545895  
     swiglu      89.488704  781.604251  
cvrp relu        80.219206  118.952592  
     swiglu      79.992027  121.698982

In [67]:
tsp_results = [get_pivot_table('tsp', N, index=['baseline']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['baseline']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50                   \
                       score       runtime           score         runtime   
     baseline                                                                
tsp  mean      3.8185 ± 0.07  1.959 ± 0.17   5.7350 ± 0.06  17.600 ± 50.30   
     val       3.8177 ± 0.07  2.067 ± 0.19   5.7451 ± 0.06  17.222 ± 37.50   
cvrp mean      6.4220 ± 0.72  3.433 ± 1.78  10.8649 ± 1.72  19.703 ± 47.38   
     val       6.4575 ± 0.72  3.558 ± 1.72  10.9066 ± 1.63  20.642 ± 41.47   

                          100                   
                        score          runtime  
     baseline                                   
tsp  mean       7.9893 ± 0.09  88.295 ± 684.96  
     val        8.0268 ± 0.09  91.003 ± 708.20  
cvrp mean      16.2839 ± 3.18  79.790 ± 117.45  
     val       16.2562 ± 3.15  80.421 ± 123.20

In [68]:
tsp_results = [get_pivot_table('tsp', N, index=['activation', 'baseline']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation', 'baseline']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50   \
                                  score       runtime           score   
     activation baseline                                                
tsp  relu       mean      3.8188 ± 0.07  2.066 ± 0.11   5.7405 ± 0.06   
                val       3.8164 ± 0.07  2.056 ± 0.11   5.7409 ± 0.06   
     swiglu     mean      3.8182 ± 0.07  1.852 ± 0.22   5.7294 ± 0.07   
                val       3.8190 ± 0.07  2.077 ± 0.27   5.7494 ± 0.06   
cvrp relu       mean      6.4137 ± 0.75  3.405 ± 1.81  10.8733 ± 1.68   
                val       6.4904 ± 0.75  3.599 ± 1.76  10.9155 ± 1.71   
     swiglu     mean      6.4303 ± 0.70  3.461 ± 1.74  10.8565 ± 1.76   
                val       6.4246 ± 0.68  3.517 ± 1.68  10.8976 ± 1.55   

                                                     100                   
                                 runtime           score          runtime  
     activation baseline                                                   
tsp  relu       mean      18.468 ± 51.29   8.0063 ± 0.10  88.817 ± 603.91  
                val       16.829 ± 31.01   8.0562 ± 0.09  90.803 ± 619.18  
     swiglu     mean      16.731 ± 49.31   7.9724 ± 0.08  87.774 ± 766.00  
                val       17.616 ± 43.98   7.9974 ± 0.08  91.204 ± 797.21  
cvrp relu       mean      19.931 ± 43.91  16.3229 ± 3.18  79.934 ± 133.44  
                val       19.953 ± 47.56  16.2900 ± 3.10  80.505 ± 104.47  
     swiglu     mean      19.475 ± 50.86  16.2449 ± 3.18  79.647 ± 101.46  
                val       21.330 ± 35.39  16.2225 ± 3.20  80.337 ± 141.93

In [69]:
tsp_results = [get_pivot_table('tsp', N, index=['ns', 'activation']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['ns', 'activation']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50   \
                              score       runtime           score   
     ns   activation                                                
tsp  0    relu        3.8491 ± 0.09  0.109 ± 0.00   5.7373 ± 0.06   
          swiglu      3.8473 ± 0.08  0.111 ± 0.00   5.7336 ± 0.07   
     100  relu        3.8155 ± 0.07  1.221 ± 0.05   5.7438 ± 0.06   
          swiglu      3.8138 ± 0.07  1.047 ± 0.38   5.7406 ± 0.07   
     500  relu        3.8107 ± 0.07  2.206 ± 0.09   5.7401 ± 0.06   
          swiglu      3.8138 ± 0.07  2.124 ± 0.22   5.7403 ± 0.06   
     1000 relu        3.8107 ± 0.07  3.732 ± 0.24   5.7398 ± 0.06   
          swiglu      3.8138 ± 0.07  3.650 ± 0.26   5.7402 ± 0.07   
cvrp 0    relu        6.4325 ± 0.78  0.131 ± 0.00  10.8342 ± 1.65   
          swiglu      6.4216 ± 0.73  0.133 ± 0.00  10.8171 ± 1.59   
     100  relu        6.4353 ± 0.76  1.400 ± 0.34  10.9101 ± 1.69   
          swiglu      6.4186 ± 0.68  1.474 ± 0.48  10.8943 ± 1.68   
     500  relu        6.4579 ± 0.75  4.093 ± 1.92  10.9159 ± 1.72   
          swiglu      6.4306 ± 0.68  4.010 ± 2.01  10.8953 ± 1.66   
     1000 relu        6.4726 ± 0.74  6.698 ± 3.98  10.9075 ± 1.73   
          swiglu      6.4361 ± 0.68  6.662 ± 3.51  10.8988 ± 1.67   
     250  relu                  NaN           NaN  10.8543 ± 1.62   
          swiglu                NaN           NaN  10.8223 ± 1.62   

                                                  100                     
                              runtime           score            runtime  
     ns   activation                                                      
tsp  0    relu           0.230 ± 0.00   8.0167 ± 0.08       0.450 ± 0.00  
          swiglu         0.236 ± 0.00   7.9556 ± 0.07       0.455 ± 0.00  
     100  relu           8.470 ± 3.64   8.0364 ± 0.10      26.566 ± 5.65  
          swiglu         8.448 ± 2.86   7.9942 ± 0.09      26.689 ± 5.11  
     500  relu         21.021 ± 41.52   8.0352 ± 0.09   104.714 ± 316.38  
          swiglu       20.274 ± 42.34   7.9911 ± 0.08   103.626 ± 379.32  
     1000 relu         32.162 ± 98.86   8.0293 ± 0.09  182.829 ± 1818.38  
          swiglu      31.267 ± 118.06   7.9840 ± 0.08  182.667 ± 2351.19  
cvrp 0    relu           0.277 ± 0.00  16.4441 ± 2.99       0.524 ± 0.00  
          swiglu         0.278 ± 0.00  16.3809 ± 3.06       0.526 ± 0.00  
     100  relu           7.206 ± 1.49  16.2876 ± 3.20      19.180 ± 2.34  
          swiglu         7.005 ± 1.65  16.2357 ± 3.22      19.331 ± 8.86  
     500  relu         25.497 ± 37.92  16.2894 ± 3.14     85.591 ± 81.61  
          swiglu       26.128 ± 33.24  16.2002 ± 3.19     87.290 ± 67.18  
     1000 relu        45.893 ± 143.02  16.2736 ± 3.15   175.735 ± 332.38  
          swiglu      47.268 ± 137.00  16.1917 ± 3.22   173.088 ± 349.91  
     250  relu           2.068 ± 0.99             NaN                NaN  
          swiglu         2.140 ± 1.23             NaN                NaN

In [70]:
tsp_results = [get_pivot_table('tsp', N, index=['ns', 'activation'], refine_to_str=False) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
tsp_results = tsp_results.reset_index()
ns_col_name = tsp_results.columns[0]
ns_col_name

('ns', '')

In [71]:
for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        corr_coef = get_pivot_table(env_type, N, index=['ns'], refine_to_str=False).reset_index().corr().loc['ns', 'score']
        print(f"{env_type}_{N}: {corr_coef}")

tsp_20: -0.6272926586781007
tsp_50: 0.30043222886605514
tsp_100: 0.3440442989041451
cvrp_20: 0.983399584164525
cvrp_50: 0.5862691196856036
cvrp_100: -0.6940766428397818


In [72]:
def corr(ns, score):
    return np.corrcoef(ns, score)[0, 1]

for N in [20, 50, 100]:
    ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
    score = tsp_results.loc[:, (N, 'score')]
    print(f"N={N}: ", corr(ns, score))


N=20:  -0.6251495964741132
N=50:  0.2680861099919338
N=100:  0.14696305829112608


C:\Users\admin\AppData\Local\Temp\ipykernel_18312\1406123094.py:5: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
C:\Users\admin\AppData\Local\Temp\ipykernel_18312\1406123094.py:5: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
C:\Users\admin\AppData\Local\Temp\ipykernel_18312\1406123094.py:5: FutureWarning: The behavi

In [103]:
def load_model_str_format(act, base, env_type, N):
    str_mcts = f"../result_summary/mcts/diff-0.75/pretrained_result/{env_type}/N_{N}-B_64/shared_mha-128-6-32-4-{act}-10-0.0001/1562-1-{base}"
    str_am = f"../result_summary/am/pretrained_result/{env_type}/N_{N}-B_64/shared_mha-128-6-32-4-{act}-10-0.0001/1562-1-{base}"
    return str_mcts, str_am

all_prob_result = {}

for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        indiv_result_dict = {'method': [], 'score': [], 'prob_num': []}
        for baseline in ['mean', 'val']:
            for act in ['relu', 'swiglu']:
                str_mcts, str_am = load_model_str_format(act, baseline, env_type, N)
                
                mcts_path = Path(str_mcts)
                
                for file in mcts_path.glob("**/*.json"):              
                    if 'epoch=299' in file.name:
                        result_loaded = load_json(str(file))
                        # find the number in file.name which comes after 'ns='
                        ns_num = int(str(file).split('\\')[9].split('-')[0].split('_')[1])
                        method = f"mcts-{ns_num}-{act}-{baseline}"    
                        
                        if ns_num == 250:
                            continue
                          
                        for prob_num, val in result_loaded.items():
                            if prob_num == 'average' or prob_num == 'std':
                                continue
                            
                            indiv_result_dict['prob_num'].append(prob_num)
                            indiv_result_dict['method'].append(method)
                            indiv_result_dict['score'].append(val['score'])

                    else:
                        continue
                    
                for file in Path(str_am).glob("**/*.json"):
                    if 'epoch=299' in file.name:
                        result_loaded = load_json(str(file))
                        method = f"am-{act}-{baseline}"
                        
                        for prob_num, val in result_loaded.items():
                            if prob_num == 'average' or prob_num == 'std':
                                continue
                            
                            indiv_result_dict['prob_num'].append(prob_num)
                            indiv_result_dict['method'].append(method)
                            indiv_result_dict['score'].append(val['score'])
                    else:
                        continue
                    
        all_prob_result[f"{env_type}_{N}"] = pd.DataFrame.from_dict(indiv_result_dict).pivot_table(index=['prob_num'], columns=['method'], values=['score'])

all_prob_result

{'tsp_20':                 score                                           \
 method   am-relu-mean am-relu-val am-swiglu-mean am-swiglu-val   
 prob_num                                                         
 0            3.649399    3.663830       3.649399      3.663830   
 1            3.763137    3.761996       3.761996      3.761996   
 10           3.778760    3.791715       3.778760      3.793402   
 11           3.770418    3.771767       3.771767      3.771767   
 12           4.076561    4.087908       4.076561      4.076561   
 ...               ...         ...            ...           ...   
 95           3.719823    3.674711       3.674710      3.674710   
 96           3.423659    3.423659       3.423659      3.423659   
 97           3.616749    3.603108       3.603108      3.603108   
 98           3.859744    3.864937       3.864937      3.864937   
 99           3.777364    3.777364       3.777364      3.777364   
 
                                                  

In [105]:
# define a paired t-test function for two pandas series
from scipy.stats import ttest_rel
def paired_ttest(s1, s2):
    return ttest_rel(s1, s2)


In [122]:
from itertools import combinations

p_value_low_results = {}

for problem, df in all_prob_result.items():
    problem_p_value = {}
    
    for col1, col2 in combinations(df.columns, 2):
        split_col1 = col1[1].split("-")
        split_col2 = col2[1].split("-")
        
        # if split_col1[0] != split_col2[0] and split_col1[-1] == split_col2[-1] and split_col1[-2] == split_col2[-2]:
        #     problem_p_value[col1[1], col2[1]] = paired_ttest(df[col1], df[col2]).pvalue
        
        if split_col1[0] != split_col2[0]:
            problem_p_value[col1[1], col2[1]] = paired_ttest(df[col1], df[col2]).pvalue
    
    # sort the problem_p_value by the p-value
    problem_p_value = {k: v for k, v in sorted(problem_p_value.items(), key=lambda item: item[1])}    
    
    # leave only 1 lowest p-value records from problem_p_value
    first_record = list(problem_p_value.items())[0]
    problem_p_value = {first_record[0]: round(first_record[1], 4)}
    
    # problem_p_value = {k: v for k, v in problem_p_value.items() if v < 0.05}
    
    p_value_low_results[problem] = problem_p_value

p_value_low_results    
    

{'tsp_20': {('am-swiglu-val', 'mcts-1000-swiglu-val'): 0.1},
 'tsp_50': {('am-relu-val', 'mcts-500-relu-val'): 0.0144},
 'tsp_100': {('am-relu-val', 'mcts-100-swiglu-mean'): 0.0},
 'cvrp_20': {('am-swiglu-val', 'mcts-1000-swiglu-val'): 0.0066},
 'cvrp_50': {('am-relu-val', 'mcts-100-swiglu-mean'): 0.0064},
 'cvrp_100': {('am-swiglu-val', 'mcts-100-relu-mean'): 0.0056}}